# Fonction d'entrainement et de deploiement d'un modèle LUIS

In [ ]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

import credentials

In [ ]:
with open('./data/train_utterances.json', 'r') as f:
    utterances = json.load(f)

In [ ]:
def booking_app(subscription_key):

    authoringKey = credentials.AUTHORING_KEY
    authoringEndpoint = credentials.AUTHORING_ENDPOINT
    predictionKey = credentials.PREDICTION_KEY
    predictionEndpoint = credentials.PREDICTION_KEY

    # We use a UUID to avoid name collisions.
    appName = "FlightBooking " + str(uuid.uuid4())
    version_id = "0.1"

    client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

    # Define app basics
    appDefinition = ApplicationCreateObject (name=appName, initial_version_id=version_id, culture='en-us')

    # Create app
    app_id = client.apps.add(appDefinition)

    # Get app id - necessary for all other changes
    print("Created LUIS app with ID {}".format(app_id))

    # Add intent
    print("Adding intent")
    intents = ['inform',]
    for intentName in intents:
        client.model.add_intent(app_id, version_id, intentName)

    # Add prebuilt entity
    print("Adding prebuilt entities")
    client.model.add_prebuilt(app_id, version_id, prebuilt_extractor_names=["number"])
    client.model.add_prebuilt(app_id, version_id, prebuilt_extractor_names=["geographyV2"])
    client.model.add_prebuilt(app_id, version_id, prebuilt_extractor_names=["datetimeV2"])


    # Add entity to app
    print("Adding ML entities")
    ml_entities = ["or_city", "dst_city","str_date","end_date", "budget"]
    entity_ids = {}
    for entity in ml_entities:
        entity_ids[entity] = client.model.add_entity(app_id, version_id, name=entity)

    # Add utterance examples
    print("Adding utterances")
    for i in range(len(utterances)):
        client.examples.add(app_id, version_id, utterances[i])

    # Train the app
    print("Training the app")
    async_training = client.train.train_version(app_id, version_id)
    is_trained = async_training.status == "UpToDate"
    trained_status = ["UpToDate", "Success"]
    while not is_trained:
        time.sleep(1)
        status = client.train.get_status(app_id, version_id)
        is_trained = all(m.details.status in trained_status for m in status)

    # Publish the app
    print("Publishing the app")

    publish_result = client.apps.publish(app_id,
                     version_id=version_id,
                     is_staging=False,
                     region='westeurope')

    endpoint = publish_result.endpoint_url + \
        "?subscription-key=" + subscription_key + "&q="

    print("Your app is published. You can now go to test it on\n{}".format(endpoint))

    return(app_id, version_id, client, entity_ids, endpoint)

app_id, version_id, client, entity_ids, endpoint = booking_app(credentials.SUBSCRIPTION_KEY)